# SAR and Optical Data Fusion for Remote Sensing

This notebook demonstrates how to combine Sentinel-1 (SAR) and Sentinel-2 (optical) imagery to create a fused dataset for enhanced remote sensing analysis using `rasterio`, `geopandas`, `numpy`, and `sentinelhub` in Python. SAR data provides all-weather imaging capabilities, complementing optical data for applications like land cover monitoring, flood detection, or vegetation analysis under cloudy conditions.

## Prerequisites
- Install required libraries: `rasterio`, `geopandas`, `numpy`, `matplotlib`, `sentinelhub`, `scipy` (listed in `requirements.txt`).
- Preprocessed Sentinel-1 and Sentinel-2 GeoTIFF files (e.g., from `21_download_data.ipynb` or `24_advanced_preprocessing.ipynb`).
- A GeoJSON or shapefile defining the area of interest (AOI) (e.g., `aoi.geojson`).
- Replace file paths with your own data.

## Learning Objectives
- Preprocess Sentinel-1 SAR data (e.g., speckle filtering).
- Align Sentinel-1 and Sentinel-2 data to a common CRS and resolution.
- Create a fused feature stack combining SAR and optical bands.
- Visualize and save the fused dataset for downstream analysis.

In [ ]:
# Import required libraries
import rasterio
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from rasterio.warp import reproject, Resampling
from rasterio.mask import mask
from scipy.ndimage import uniform_filter
from pyproj import CRS
import os

## Step 1: Load Sentinel-1 and Sentinel-2 Data

Load preprocessed Sentinel-1 (SAR) and Sentinel-2 (optical) GeoTIFFs and the AOI vector data.

In [ ]:
# Define file paths
s1_path = 'remote_sensing_data/sentinel1_vv_vh.tif'  # Replace with your Sentinel-1 GeoTIFF (VV, VH bands)
s2_path = 'remote_sensing_data/sentinel_rgb.tif'    # Replace with your Sentinel-2 GeoTIFF
aoi_path = 'aoi.geojson'                            # Replace with your AOI file

# Load Sentinel-1 data
with rasterio.open(s1_path) as src_s1:
    s1_data = src_s1.read(masked=True)  # Shape: (bands, height, width) (e.g., VV, VH)
    s1_profile = src_s1.profile
    s1_crs = src_s1.crs

# Load Sentinel-2 data
with rasterio.open(s2_path) as src_s2:
    s2_data = src_s2.read(masked=True)  # Shape: (bands, height, width)
    s2_profile = src_s2.profile
    s2_crs = src_s2.crs

# Load AOI
aoi_gdf = gpd.read_file(aoi_path)
if aoi_gdf.crs != s2_crs:
    aoi_gdf = aoi_gdf.to_crs(s2_crs)

# Print basic information
print(f'Sentinel-1 shape: {s1_data.shape}, CRS: {s1_crs}')
print(f'Sentinel-2 shape: {s2_data.shape}, CRS: {s2_crs}')
print(f'AOI CRS: {aoi_gdf.crs}')

## Step 2: Preprocess Sentinel-1 SAR Data

Apply speckle filtering to reduce noise in Sentinel-1 SAR imagery.

In [ ]:
# Apply Lee filter for speckle noise reduction
def lee_filter(img, size=5):
    img_mean = uniform_filter(img, size=size)
    img_sqr = img ** 2
    img_sqr_mean = uniform_filter(img_sqr, size=size)
    img_variance = img_sqr_mean - img_mean ** 2
    overall_variance = np.var(img)
    img_weight = img_variance / (img_variance + overall_variance)
    return img_mean + img_weight * (img - img_mean)

# Process Sentinel-1 bands (VV, VH)
filtered_s1_data = np.zeros_like(s1_data, dtype=np.float32)
for i in range(s1_data.shape[0]):
    filtered_s1_data[i] = lee_filter(s1_data[i], size=5)

# Visualize filtered Sentinel-1 VV band
plt.figure(figsize=(8, 8))
plt.imshow(filtered_s1_data[0], cmap='gray', vmin=np.nanpercentile(filtered_s1_data[0], 2), vmax=np.nanpercentile(filtered_s1_data[0], 98))
plt.title('Filtered Sentinel-1 VV Band')
plt.xlabel('Column')
plt.ylabel('Row')
plt.colorbar(label='Backscatter')
plt.show()

## Step 3: Align Sentinel-1 to Sentinel-2 CRS and Resolution

Reproject and resample Sentinel-1 data to match Sentinel-2's CRS and resolution (10m).

In [ ]:
# Define target CRS and resolution (use Sentinel-2 as reference)
target_crs = s2_crs
target_transform = s2_profile['transform']
target_width, target_height = s2_profile['width'], s2_profile['height']

# Reproject Sentinel-1 data
aligned_s1_data = np.empty((filtered_s1_data.shape[0], target_height, target_width), dtype=np.float32)
with rasterio.open(s1_path) as src:
    for i in range(filtered_s1_data.shape[0]):
        reproject(
            source=filtered_s1_data[i],
            destination=aligned_s1_data[i],
            src_transform=s1_profile['transform'],
            src_crs=s1_crs,
            dst_transform=target_transform,
            dst_crs=target_crs,
            resampling=Resampling.bilinear
        )

# Update Sentinel-1 profile
aligned_s1_profile = s2_profile.copy()
aligned_s1_profile.update({'count': filtered_s1_data.shape[0]})

# Visualize aligned Sentinel-1 VV band
plt.figure(figsize=(8, 8))
plt.imshow(aligned_s1_data[0], cmap='gray', vmin=np.nanpercentile(aligned_s1_data[0], 2), vmax=np.nanpercentile(aligned_s1_data[0], 98))
plt.title('Aligned Sentinel-1 VV Band')
plt.xlabel('Column')
plt.ylabel('Row')
plt.colorbar(label='Backscatter')
plt.show()

## Step 4: Crop to AOI

Crop both Sentinel-1 and Sentinel-2 data to the AOI for consistent analysis.

In [ ]:
# Crop Sentinel-2 to AOI
with rasterio.open(s2_path) as src:
    cropped_s2_data, cropped_s2_transform = mask(src, aoi_gdf.geometry, crop=True, nodata=np.nan)
cropped_s2_profile = s2_profile.copy()
cropped_s2_profile.update({
    'height': cropped_s2_data.shape[1],
    'width': cropped_s2_data.shape[2],
    'transform': cropped_s2_transform,
    'nodata': np.nan
})

# Crop aligned Sentinel-1 to AOI
cropped_s1_data = np.zeros((aligned_s1_data.shape[0], cropped_s2_data.shape[1], cropped_s2_data.shape[2]), dtype=np.float32)
temp_s1_path = 'temp_s1_aligned.tif'
with rasterio.open(temp_s1_path, 'w', **aligned_s1_profile) as dst:
    dst.write(aligned_s1_data)
with rasterio.open(temp_s1_path) as src:
    cropped_s1_data, cropped_s1_transform = mask(src, aoi_gdf.geometry, crop=True, nodata=np.nan)

# Update Sentinel-1 profile
cropped_s1_profile = cropped_s2_profile.copy()
cropped_s1_profile.update({'count': cropped_s1_data.shape[0]})

# Visualize cropped datasets
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
ax1.imshow(cropped_s2_data[:3].transpose(1, 2, 0) / np.nanpercentile(cropped_s2_data[:3], 98), vmin=0, vmax=1)
ax1.set_title('Cropped Sentinel-2 RGB')
ax1.set_xlabel('Column')
ax1.set_ylabel('Row')
ax2.imshow(cropped_s1_data[0], cmap='gray', vmin=np.nanpercentile(cropped_s1_data[0], 2), vmax=np.nanpercentile(cropped_s1_data[0], 98))
ax2.set_title('Cropped Sentinel-1 VV')
ax2.set_xlabel('Column')
ax2.set_ylabel('Row')
plt.tight_layout()
plt.show()

## Step 5: Create Fused Feature Stack

Combine Sentinel-1 and Sentinel-2 bands into a single feature stack.

In [ ]:
# Normalize datasets for consistent scaling
norm_s2_data = cropped_s2_data / np.nanpercentile(cropped_s2_data, 98, axis=(1, 2), keepdims=True)
norm_s2_data = np.clip(norm_s2_data, 0, 1)
norm_s1_data = cropped_s1_data / np.nanpercentile(cropped_s1_data, 98, axis=(1, 2), keepdims=True)
norm_s1_data = np.clip(norm_s1_data, 0, 1)

# Combine into fused feature stack
fused_data = np.concatenate([norm_s2_data, norm_s1_data], axis=0)

# Update profile for fused data
fused_profile = cropped_s2_profile.copy()
fused_profile.update({'count': fused_data.shape[0]})

# Visualize first three bands of fused stack (Sentinel-2 RGB)
fused_rgb = fused_data[:3].transpose(1, 2, 0)
fused_rgb = np.clip(fused_rgb, 0, 1)

plt.figure(figsize=(8, 8))
plt.imshow(fused_rgb)
plt.title('Fused Feature Stack (Sentinel-2 RGB)')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

# Save fused feature stack
fused_output_path = 'remote_sensing_data/sar_optical_fused.tif'
with rasterio.open(fused_output_path, 'w', **fused_profile) as dst:
    dst.write(fused_data)

print(f'Fused feature stack saved to: {fused_output_path}')

## Step 6: Validate with AOI Overlay

Overlay the AOI on the fused data to verify alignment.

In [ ]:
# Visualize fused data with AOI overlay
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(fused_rgb)
aoi_gdf.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2)
plt.title('Fused Feature Stack with AOI Overlay')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

# Clean up temporary files
os.remove(temp_s1_path)

## Next Steps

- Replace `sentinel1_vv_vh.tif` and `sentinel_rgb.tif` with your own Sentinel-1 and Sentinel-2 GeoTIFFs (e.g., from `21_download_data.ipynb` or `24_advanced_preprocessing.ipynb`).
- Update `aoi.geojson` with your area of interest file.
- Experiment with advanced fusion methods (e.g., deep learning-based fusion) for improved results.
- Use the fused feature stack in downstream tasks like classification (see `12_classification_rf_svm.ipynb` or `27_transfer_learning.ipynb`) or segmentation (see `15_unet_segmentation.ipynb`).
- Explore visualization of the fused data with `23_kepler_gl_demo.ipynb` or `26_time_series_animation.ipynb`.

## Notes
- Ensure Sentinel-1 and Sentinel-2 data are acquired from similar time periods to avoid temporal misalignment.
- Speckle filtering parameters (e.g., filter size) can be adjusted based on image characteristics.
- Normalize data carefully to maintain consistency across SAR and optical bands.
- See `docs/installation.md` for troubleshooting library installation.